# Processing the dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import roslib
import tf
from memmo_utils import *
from database import *
import os
from geometry_msgs.msg import Quaternion, Pose, Point, Vector3
from std_msgs.msg import Header, ColorRGBA
import rospy
import os
from mlp.utils.status import Status
import mlp.utils.wholebody_result as wb_res
import matplotlib.pyplot as plt
import pickle
from IPython.display import clear_output

from transforms3d import axangles
from transforms3d import affines, quaternions

from regression import *

In [2]:
np.set_printoptions(precision=4, suppress=True)

%load_ext autoreload
%autoreload 2

### Start Instruction

### Load Robot Kinematics using URDF 

In [3]:
import roslib; roslib.load_manifest('urdfdom_py')
from urdf_parser_py.urdf import URDF
from pykdl_utils.kdl_parser import kdl_tree_from_urdf_model
from pykdl_utils.kdl_kinematics import KDLKinematics

robot = URDF.from_parameter_server()
tree = kdl_tree_from_urdf_model(robot)

left_foot = KDLKinematics(robot, 'base_link', 'left_sole_link')
right_foot = KDLKinematics(robot, 'base_link', 'right_sole_link')

q = left_foot.random_joint_angles()
pose = left_foot.forward(q) # forward kinematics (returns homogeneous 4x4 numpy.mat)
q_ik = left_foot.inverse(pose, q+0.3) # inverse kinematics
clear_output()

### Create Visualization

In [ ]:
viz = Visual()
viz.rate = rospy.Rate(1000)

foot_marker = RvizMarker('foot_marker', 5, Marker.CUBE, 4)
colors = [ColorRGBA(0.0, 1.0, 0.0, 0.3), ColorRGBA(0.0, 0.0, 1.0, 0.3)] + [ColorRGBA(1.0, 0.0, 0.0, 0.9)]*2
foot_marker.set_color(colors)

### 1. Load and store raw_data

In [4]:
FILE_CAT = 'talos_moveEffector_flat'
#FILE_CAT = 'talos_circle'
#FILE_CAT = 'talos_circle_oriented'
ROOT = '/media/teguh/Data/MEMMO Dataset/' + FILE_CAT + '/'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

file_names = os.listdir(ROOT)
foot_poses = []
trajs = []
vel_trajs = []
u_trajs = []
stats = []
data_phases_set = []
res_set = []
T_lefts_set = []
T_rights_set = []

to_play = False
n_high_accels = 0

for i in range(100):
#for i in range(len(file_names)):
    print i
    filename = file_names[i]
    try:
        traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t, stat, \
        res = extract_from_dataset(ROOT+filename, FILE_CAT, False, 1, version='v1.0')
    except:
        print 'Fail'
        continue 
        
    acc = res.ddq_t.T
    if np.abs(np.max(acc[:100,0])) > 1.:
        print filename + ' has high accelerations!'
        n_high_accels += 1
        continue
        #foot_marker.publish(foot_pose)
        #viz.vis_traj(traj)
            
    #move the root so that the sole_link is at 0. instead of 0.004
    T = affines.compose(np.array([0.,0.,-0.004]), np.eye(3), np.ones(3))
    traj = transform_traj(traj,T)
    traj = np.vstack(traj)
    
    foot_pose = construct_foot_poses(T_lefts, T_rights,False)

    if to_play:
        foot_marker.publish(foot_pose)
        viz.vis_traj(traj)
        

    foot_poses.append(foot_pose)
    trajs.append(traj)
    vel_trajs.append(vel_traj)
    u_trajs.append(u_traj)
    data_phases_set.append(data_phases)
    stats.append(stat)
    res_set.append(res)
    T_lefts_set.append(T_lefts)
    T_rights_set.append(T_rights)

clear_output()
print 'Saving data....'

data = dict()
data['foot_poses'] = np.array(foot_poses)
data['trajs'] = np.array(trajs)
data['vel_trajs'] = np.array(vel_trajs)
data['u_trajs'] = np.array(u_trajs)
data['data_phases_set'] = np.array(data_phases_set)
data['stats']  = np.array(stats)
data['T_lefts_set']  = T_lefts_set
data['T_rights_set']  = T_rights_set
data['information'] = FILE_CAT + ', Raw data'

f = open(ROOT_PROCESSED + 'raw_data_sample100.pkl', 'wb')
pickle.dump(data,f)
f.close()
print 'Data has been saved!'

Saving data....
Data has been saved!


### 2. One Step Movement

#### Load data and transform the root to zero positions 

In [ ]:
FILE_CAT = 'talos_moveEffector_flat'
ROOT = '/media/teguh/Data/MEMMO Dataset/' + FILE_CAT + '/'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

file_names = os.listdir(ROOT)
foot_poses = []
trajs = []
vel_trajs = []
u_trajs = []
stats = []
data_phases_set = []
res_set = []
T_lefts_set = []
T_rights_set = []

to_play = False
n_high_accels = 0
#for i in range(0,10):
for i in range(len(file_names)):
    print i
    filename = file_names[i]
    try:
        traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t, stat, \
        res = extract_from_dataset(ROOT+filename, FILE_CAT, False, 1)
    except:
        print 'Fail'
        continue 
        
    acc = res.ddq_t.T
    if np.abs(np.max(acc[:100,0])) > 1.:
        print filename + ' has high accelerations!'
        n_high_accels += 1
        continue
        
    #transform the trajectory to start from zero in the base frame
    #and transform the quaternion to transforms3d format
    Tw0 = calc_root_pose(traj[0,0:7])
    #affines.compose(traj[0,0:3], quaternions.quat2mat(rectify_quat(traj[0,3:7])), np.ones(3))
    Tw0[2,3] = -0.004 #to offset the solelink down by 4mm
    Tw0_inv = np.linalg.inv(Tw0)
    traj =  transform_traj(traj, Tw0_inv)
    
    T_lefts = transform_SE3s(T_lefts, Tw0_inv)
    T_rights = transform_SE3s(T_rights, Tw0_inv)
    foot_pose = construct_foot_poses(T_lefts, T_rights,False)

    
    if to_play:
        foot_marker.publish(foot_pose)
        viz.vis_traj(traj)

    foot_poses.append(foot_pose)
    trajs.append(traj)
    vel_trajs.append(vel_traj)
    u_trajs.append(u_traj)
    data_phases_set.append(data_phases)
    stats.append(stat)
    res_set.append(res)
    T_lefts_set.append(T_lefts)
    T_rights_set.append(T_rights)

clear_output()
print 'Saving data....'

data = dict()
data['foot_poses'] = np.array(foot_poses)
data['trajs'] = np.array(trajs)
data['vel_trajs'] = np.array(vel_trajs)
data['u_trajs'] = np.array(u_trajs)
data['data_phases_set'] = np.array(data_phases_set)
data['stats']  = np.array(stats)
data['T_lefts_set']  = T_lefts_set
data['T_rights_set']  = T_rights_set
data['information'] = 'One step movement. The trajectories are transformed s.t. the \
root positions start from zero. '
f = open(ROOT_PROCESSED + 'data.pkl', 'wb')
pickle.dump(data,f)
f.close()
print 'Data has been saved!'

#### Load data and transform the root to zero positions, and separate into left and right data

In [ ]:
FILE_CAT = 'talos_moveEffector_flat'
ROOT = '/media/teguh/Data/MEMMO Dataset/' + FILE_CAT + '/'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

file_names = os.listdir(ROOT)

data = dict()
data['left'] = dict()
data['right'] = dict()
for key in data.keys():
    data[key]['foot_poses'] = []
    data[key]['trajs'] = []
    data[key]['vel_trajs'] = []
    data[key]['u_trajs'] = []
    data[key]['stats'] = []
    data[key]['data_phases_set'] = []
    data[key]['res_set'] = []
    data[key]['T_lefts_set'] = []
    data[key]['T_rights_set'] = []
    data[key]['x_inputs'] = []
    data[key]['x_inputs_root'] = []

to_play = False
n_high_accels = 0
#for i in range(0,500):
for i in range(len(file_names)):
    print i
    filename = file_names[i]
    try:
        traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t, stat, \
        res = extract_from_dataset(ROOT+filename, FILE_CAT, False, 1)
    except:
        print 'Fail'
        continue 
    
    
    acc = res.ddq_t.T
    if np.abs(np.max(acc[:100,0])) > 1.:
        print filename + ' has high accelerations!'
        n_high_accels += 1
        continue
        
    #transform the trajectory to start from zero in the base frame
    #and transform the quaternion to transforms3d format
    Tw0 = calc_root_pose(traj[0,0:7])
    #affines.compose(traj[0,0:3], quaternions.quat2mat(rectify_quat(traj[0,3:7])), np.ones(3))
    #Tw0[2,3] = -0.004 #to offset the solelink down by 4mm
    Tw0[2,3] = 0.
    Tw0_inv = np.linalg.inv(Tw0)
    traj =  transform_traj(traj, Tw0_inv)
    
    T_lefts = transform_SE3s(T_lefts, Tw0_inv)
    T_rights = transform_SE3s(T_rights, Tw0_inv)
    foot_pose = construct_foot_poses(T_lefts, T_rights,False)
    
    if to_play:
        foot_marker.publish(foot_pose)
        viz.vis_traj(traj)
        
        
    key = determine_which_foot(foot_pose)
    if key == 'left':
        x_input = np.concatenate([foot_pose[0:2].flatten(), foot_pose[2]])
    elif key == 'right':
        x_input = np.concatenate([foot_pose[0:2].flatten(), foot_pose[3]])
    else:
        print cur_foot_pose
        break

    x_input_root = np.concatenate([foot_pose[0:2].flatten(), traj[-1][0:7]])

    data[key]['foot_poses'].append(foot_pose)
    data[key]['trajs'].append(traj)
    data[key]['vel_trajs'].append(vel_traj)
    data[key]['u_trajs'].append(u_traj)
    data[key]['x_inputs'].append(x_input)
    data[key]['x_inputs_root'].append(x_input_root)
    data[key]['data_phases_set'].append(data_phases)
    

clear_output()
print 'Saving data....'

for key in data.keys():
    for key2 in data[key].keys():
        data[key][key2] = np.array(data[key][key2])

data['information'] = 'One step movement, created from ' + FILE_CAT + '. The trajectories are transformed s.t. the \
root positions start from zero. The data are separated into right and left'

f = open(ROOT_PROCESSED + 'data_left_right.pkl', 'wb')
pickle.dump(data,f)
print 'Data has been saved!'
f.close()

In [ ]:
data[key]['trajs'].shape

### 3. Talos_circle/Talos_circle_oriented 

#### Separate into one-step movements and Transform the root into zero

In [6]:
#FILE_CAT = 'talos_circle'
FILE_CAT = 'talos_circle_oriented'
ROOT = '/media/teguh/Data/MEMMO Dataset/' + FILE_CAT + '/'
ROOT_PROCESSED = '/media/teguh/Data/MEMMO Dataset/processed_data/' + FILE_CAT + '/'

file_names = os.listdir(ROOT)

time_step = 2500

data = dict()
data['left'] = dict()
data['right'] = dict()
for key in data.keys():
    data[key]['foot_poses'] = []
    data[key]['trajs'] = []
    data[key]['vel_trajs'] = []
    data[key]['u_trajs'] = []
    data[key]['stats'] = []
    data[key]['data_phases_set'] = []
    data[key]['res_set'] = []
    data[key]['T_lefts_set'] = []
    data[key]['T_rights_set'] = []
    data[key]['x_inputs'] = []
    data[key]['x_inputs_root'] = []
    
to_play = False
n_high_accels = 0

for i in range(len(file_names)):
    print i
    filename = file_names[i]
    try:
        traj, vel_traj, u_traj, T_lefts, T_rights, data_phases, delta_t, stat, \
        res = extract_from_dataset(ROOT+filename, FILE_CAT, False, 1)
    except:
        print 'Fail'
        continue 
        
    acc = res.ddq_t.T
    if np.abs(np.max(acc[:100,0])) > 1.:
        print filename + ' has high accelerations!'
        n_high_accels += 1
        continue
        
    #move the root so that the sole_link is at 0. instead of 0.004
    T = affines.compose(np.array([0.,0.,-0.004]), np.eye(3), np.ones(3))
    traj = transform_traj(traj,T)
   
    #Set the height of the foot to zero
    for T in T_lefts: 
        if T is not None: T[2,3] = 0.00
    for T in T_rights: 
        if T is not None: T[2,3] = 0.00
            
    foot_pose = construct_foot_poses(T_lefts, T_rights,False)
    
    #separate the data according to the phase
    n_phase = len(data_phases)
    for i in range((n_phase-1)/2):
        len_bef = len(data_phases[2*i])
        len_aft = len(data_phases[2*i+2])
        
        cur_phase = np.concatenate([data_phases[2*i][len_bef/2:], data_phases[2*i+1], data_phases[2*i+2][:len_aft/2]])
        cur_traj = traj[cur_phase]

        cur_foot_pose = foot_pose[2*i:2*i+4].copy()
        
        #transform according to the initial root configuration
        Tw0 = calc_root_pose(cur_traj[0,0:7])
        Tw0[2,3] = 0
        Tw0_inv = np.linalg.inv(Tw0)
        cur_traj =  transform_traj(cur_traj, Tw0_inv)
        cur_traj = np.vstack(cur_traj)
        cur_traj = subsample(cur_traj, time_step)
        
        #transform foot_pose to zero root position
        for j in range(len(cur_foot_pose)):
            T = PosetoMat(cur_foot_pose[j]) 
            T = np.dot(Tw0_inv, T)
            cur_foot_pose[j] = MattoPose(T)
            
        key = determine_which_foot(cur_foot_pose)
        if key == 'left':
            x_input = np.concatenate([cur_foot_pose[0:2].flatten(), cur_foot_pose[2]])
        elif key == 'right':
            x_input = np.concatenate([cur_foot_pose[0:2].flatten(), cur_foot_pose[3]])
        else:
            print cur_foot_pose
            break
            
        x_input_root = np.concatenate([cur_foot_pose[0:2].flatten(), cur_traj[-1][0:7]])
            
        data[key]['foot_poses'].append(cur_foot_pose)
        data[key]['trajs'].append(cur_traj)
        data[key]['x_inputs'].append(x_input)
        data[key]['x_inputs_root'].append(x_input_root)
        
        if to_play:
            foot_marker.publish(cur_foot_pose)
            viz.vis_traj(cur_traj)

    if key == 'Cannot determine which foot!':
        break
        
clear_output()
print 'Saving data....'

for key in data.keys():
    for key2 in data[key].keys():
        data[key][key2] = np.array(data[key][key2])

data['information'] = 'One step movement, created from ' + FILE_CAT + '. The trajectories are transformed s.t. the \
root positions start from zero. The data are separated into right and left'
f = open(ROOT_PROCESSED + 'data_left_right.pkl', 'wb')
pickle.dump(data,f)
f.close()
print 'Data has been saved!'

Saving data....
Data has been saved!


In [ ]:
f = open('data/' + FILE_CAT + '/data_left_right.pkl', 'wb')
pickle.dump(data,f)
f.close()

In [ ]:
trajs_left = data['left']['trajs']
foot_poses = data['left']['foot_poses']

In [ ]:
for i in range(10):
    foot_marker.publish(foot_poses[i])
    viz.vis_traj(trajs_left[i])